LLMs can be used for a variety of tasks, but they are often limited in what they can do on their own. For simple tasks or applications, we can use LLMs in isolation. But for more complex tasks, it is often necessary to chain or combine different components. 

**Chains** in LangChain are a series of steps and actions.

**Chains** allows us to combine multiple components together to solve a specific task and build an entire LLM application.

LangChain provides a standard interface for chains, as well as some common implementations of chains to make it easy to create and use chains.

There are:
1)Simple Chains
2)Sequential Chains
3)Custom Chains

In [2]:
pip install -r ./requirements.txt -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
#load the environment variables.

import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)   

True

# 1)Simple Chains
It consists of a single LLM that is used to perform a single task. Simple chains are easy to create and use and also relatively efficient. However, they are limited in what they can do. 
for more complex tasks, it is often necessary to use sequential chains.

In [6]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = ''' You are an experienced virologist.
Write a few sentences about the following virus {virus} in {language}.'''

prompt_template = PromptTemplate.from_template(template=template)


#Create the chain by calling LLMChain() constructor.This constructor sets up a simple chain and takes the prompt template, fills it with user input and returns the response from the LLM.

chain = LLMChain(
    llm=llm,
    prompt = prompt_template, 
    #If you like to run the chain in verbose mode to display intermediate logs, then add "verbose=True" to the chain constructor 
    verbose=True
)

output= chain.invoke({'virus':'covid', 'language': 'english'})
print(output)



> Entering new LLMChain chain...
Prompt after formatting:
 You are an experienced virologist.
Write a few sentences about the following virus covid in english.

> Finished chain.
{'virus': 'covid', 'language': 'english', 'text': 'COVID-19, also known as the coronavirus, is a highly contagious virus that primarily spreads through respiratory droplets. It belongs to the family of coronaviruses and has caused a global pandemic since its emergence in late 2019. The virus has led to widespread illness, hospitalizations, and deaths, prompting governments and health organizations to implement strict measures to control its spread. Research and vaccination efforts are ongoing to combat the virus and protect public health.'}


**Note**:-In the above example, the prompt template requires two variables, but if it requires only one variable then we can call "chain.invoke()" with a string variable as an argument instead of a dict. For detail see below example:

In [8]:
template = 'What is the capital of {country}? List the top 3 places to visit in that city. Use bullet points.'
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt = prompt_template, 
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
#Here "output" is a dictionary and "text" is a key of the dictionary. The value of the text key contains the response.
print(output['text'])

Enter Country:  canada




> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of canada? List the top 3 places to visit in that city. Use bullet points.

> Finished chain.
The capital of Canada is Ottawa.

Top 3 places to visit in Ottawa:

- Parliament Hill
- National Gallery of Canada
- ByWard Market


# 2)Sequential Chains
With **Sequential chains**, you can make a series of calls to one or more LLMs. You can take the output from one chain and use it as the input to another chain. 

There are two types of sequential chains:
**1)SimpleSequentialChain
2)General form of sequential chains**

**SimpleSequentialChain** represents a series of chains, where each individual chain has a single input and a single output, and the output of one step is used as input to the next.


In [9]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Node programmer. Write a code that implements the concept of {concept}.'
)
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
#llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the node code {code}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)


overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output = overall_chain.invoke('API')

print(output['output'])



> Entering new SimpleSequentialChain chain...
Sure, here is an example of a simple API implemented using Node.js:

```javascript
// Import required modules
const express = require('express');

// Create an instance of Express
const app = express();
const PORT = 3000;

// Define a sample data object
const data = {
  name: 'John Doe',
  age: 30,
  occupation: 'Scientist'
};

// Define a route to retrieve the sample data
app.get('/data', (req, res) => {
  res.json(data);
});

// Start the server
app.listen(PORT, () => {
  console.log(`Server is running on http://localhost:${PORT}`);
});
```

In this code snippet, we are creating a simple API using Express, a popular Node.js framework for building web applications. We define a route `/data` that returns a JSON response containing sample data. When the server is started, it listens on port 3000 and outputs a message to the console.

To run this code, save it in a file (e.g., `api.js`) and run `node api.js` in the terminal. You can then ac